In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

tolerance = 0.00001
multim_n_consumer = 5

## 🧪 Practice 6: Solving for Mean Utilities via Contraction Mapping

### 🎯 Goal
Find the vector of mean utilities $\delta_j$ that rationalizes observed market shares $s_j^{\text{obs}}$,  
by matching simulated shares $s_j^{\text{pred}}(\delta)$ to observed shares.

---

### 🧠 Theory

Contraction mapping update rule:

$$
\delta_j^{\text{new}} = \delta_j^{\text{old}} + \log(s_j^{\text{obs}}) - \log(s_j^{\text{pred}})
$$

- Adjust $\delta_j$ by the log difference between observed and predicted shares.
- Repeat until $\delta$ converges.

---

### ⚙️ Steps

1. Initialize $\delta_j$ (e.g., zeros)
2. Given $\delta_j$, compute choice probabilities and aggregate to $s_j^{\text{pred}}$
3. Update $\delta_j$ using the contraction formula
4. Check if the change in $\delta_j$ is below a tolerance (e.g., $10^{-6}$)
5. Repeat until convergence

---

### 📈 Purpose

- Recovers the mean utilities that exactly match observed market shares.
- Forms the **core inner loop** of the BLP estimation algorithm.
- Prepares for GMM estimation of random coefficient parameters (Practice 7).

In [2]:
# Specification: define functions 

base_utility = pd.read_csv("simulated_utilities.csv")
delta_guess = [0,0,0,0,0]

dataset = pd.read_csv("multi_market_data.csv")

market = dataset[dataset['market_id'] == 0].reset_index(drop=True) #extract multi market dataset

observed_share = market['share'].values

def predicted_share(input_utilities, input_delta): #output predicted shares 
    adjusted_utilities = input_utilities.values + input_delta
    sum_utility = []
    n_consumer = adjusted_utilities.shape[0]
    n_product = adjusted_utilities.shape[1]
    for i in range(n_consumer): 
        temp = 0
        for j in range(n_product):
            temp += np.exp(adjusted_utilities[i][j])
        sum_utility.append(temp)
    
    prob_choice = adjusted_utilities.copy()
    prob_choice = np.exp(prob_choice)
    for i in range(n_consumer):
        prob_choice[i] = prob_choice[i]/(sum_utility[i]+1)

    pred_share = np.zeros(n_product)
    for i in range(n_consumer):
        for j in range(n_product):
            pred_share[j] += prob_choice[i][j]
    pred_share = pred_share/10
    return pred_share


adjusted_share = predicted_share(base_utility, delta_guess)

In [3]:
# Contraction mapping algorithm 
difference = 1
delta_guess = [0,0,0,0,0]
# we already have base utility and delta guess 

while difference > tolerance:    
    adjusted_share = predicted_share(base_utility, delta_guess)
    delta_new = delta_guess + np.log(observed_share) - np.log(adjusted_share)
    difference = np.max(np.abs(delta_new - delta_guess))
    # print(delta_guess, delta_new, difference)
    delta_guess = delta_new

print("our converged mean utilities are", delta_guess)

our converged mean utilities are [ 0.238 -0.006 -0.04   0.015  0.063]


## 🧪 Practice 7: GMM Estimation of Random Coefficients

### 🎯 Goal

Estimate the random coefficient parameter $\sigma$ (and linear parameters $\beta$, $\alpha$)  
using the BLP GMM framework with moment conditions based on unobserved product characteristics $\xi_{jt}$.

---

### 🧠 Theory

The structural demand equation is:

$$
\delta_{jt} = x_{jt} \beta - \alpha p_{jt} + \xi_{jt}
$$

- You have already recovered $\delta_{jt}$ from observed shares using contraction mapping.
- The residual $\xi_{jt}$ reflects unobserved demand shocks.
- Valid instruments $z_{jt}$ should satisfy:

$$
\mathbb{E}[z_{jt} \cdot \xi_{jt}] = 0
$$

---

### ⚙️ GMM Estimation Procedure

1. **Fix a value of** $\sigma$
2. **Simulate** random coefficients for consumers using $\sigma$
3. **Solve** for $\delta_{jt}$ using contraction mapping for each market
4. **Estimate** $(\beta, \alpha)$ from:

$$
\delta = X\theta + \xi, \quad \text{with instruments } Z
$$

5. **Compute residuals** $\xi_{jt} = \delta_{jt} - x_{jt} \beta + \alpha p_{jt}$
6. **Form moment condition vector**:

$$
\hat{g}(\sigma) = \frac{1}{N} \sum z_{jt} \cdot \xi_{jt}
$$

7. **Compute GMM loss function**:

$$
Q(\sigma) = \hat{g}(\sigma)^\top W \hat{g}(\sigma)
$$

8. **Minimize** $Q(\sigma)$ over values of $\sigma$

---

### 📌 Notes

- The outer loop optimizes $\sigma$
- The inner loop simulates demand and recovers $\delta$ using contraction mapping
- $W$ is the GMM weighting matrix (start with identity, refine iteratively)
- Instruments $z_{jt}$ can include cost shifters, characteristics of rival products, etc.

---

### 📈 Output

- Estimated heterogeneity parameter $\hat{\sigma}$
- Estimated linear parameters $\hat{\beta}$ and $\hat{\alpha}$
- Residuals $\hat{\xi}_{jt}$, used for model fit and marginal cost recovery

In [4]:
# Practice 7: Scaling it to multiple markets

# we didn't creat an instrument for our simulated dataset
# so now we will add in the instrument retroactively 

dataset = pd.read_csv("multi_market_data.csv")
dataset['z_rival_x'] = dataset.groupby('market_id')['x'].transform('sum') - dataset['x']
dataset['z_rival_x'] = dataset['z_rival_x']/multim_n_consumer



